In [1]:
import pickle
import pprint
from datetime import datetime
from pathlib import Path

import albumentations as A
import cv2
import numpy as np
import polars as pl
import pytz
import timm
import torch
import torch.nn.functional as F
from albumentations import ReplayCompose
from albumentations.pytorch import ToTensorV2
from omegaconf import OmegaConf
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
from transformers import get_cosine_schedule_with_warmup

from src.config import cfg
from src.dir import create_dir
from src.seed import seed_everything

cfg.exp_number = Path().resolve().name
print(OmegaConf.to_yaml(cfg, resolve=True))

seed_everything(cfg.seed)
pl.Config.set_fmt_str_lengths(1000)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


/home/marumarukun/pj/compe/atma_18/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


exp_number: '005'
run_time: base
data:
  input_root: ../../data/input
  train_path: ../../data/input/train_features.csv
  test_path: ../../data/input/test_features.csv
  sample_submission_path: ../../data/input/sample_submission.csv
  img_root: ../../data/input/images
  output_root: ../../data/output
  results_root: ../../results
  results_path: ../../results/005/base
seed: 319
n_splits: 5
target_cols:
- x_0
- y_0
- z_0
- x_1
- y_1
- z_1
- x_2
- y_2
- z_2
- x_3
- y_3
- z_3
- x_4
- y_4
- z_4
- x_5
- y_5
- z_5
cnn:
  model_name: tf_efficientnet_b0_ns
  size: 224
  pretrained: true
  in_chans: 3
  target_size: 18
  lr: 0.0001
  num_epochs: 20
  batch_size: 32

Using device: cuda


### exp005

- ちょっと凝ったNNモデルを作ってみるnotebook
- アーキテクチャ
    - エンコーダー: EfficientNet-B0
    - プーリング層: GeM
    - Transformerレイヤー: 1層
    - 位置エンコーディング: 1, 3, n_features
    - 全結合層: 2層


### データの読み込み

In [2]:
# データの読み込み
train = pl.read_csv(cfg.data.train_path, try_parse_dates=True)
test = pl.read_csv(cfg.data.test_path, try_parse_dates=True)
sample_submission = pl.read_csv(cfg.data.sample_submission_path, try_parse_dates=True)

# index列を作成
train = train.with_row_index()
test = test.with_row_index()

# データの結合(label encoding用)
train_test = pl.concat([train, test], how="diagonal")

# scene列を作成 → これでGroupKFoldする
train = train.with_columns(pl.col("ID").str.split("_").list[0].alias("scene"))
test = test.with_columns(pl.col("ID").str.split("_").list[0].alias("scene"))

# CV
gkf = GroupKFold(n_splits=cfg.n_splits)

# fold列を作成
train = train.with_columns(pl.lit(-1).alias("fold"))
for i, (_, valid_idx) in enumerate(gkf.split(train, groups=train["scene"])):
    fold = i + 1
    train = train.with_columns(
        pl.when(pl.col("index").is_in(valid_idx)).then(fold).otherwise(pl.col("fold")).alias("fold")
    )

train["fold"].value_counts()


fold,count
i32,u32
5,8674
3,8674
2,8674
4,8674
1,8675


In [3]:
train.head()


index,ID,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,leftBlinker,rightBlinker,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,scene,fold
u32,str,f64,f64,f64,f64,f64,bool,f64,bool,str,bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i32
0,"""00066be8e20318869c38c66be466631a_320""",5.701526,1.538456,-2.165777,-139.0,0.0,false,0.25,true,"""drive""",false,false,2.82959,0.032226,0.045187,6.231999,0.065895,0.107974,9.785009,0.124972,0.203649,13.485472,0.163448,0.302818,17.574227,0.174289,0.406331,21.951269,0.199503,0.485079,"""00066be8e20318869c38c66be466631a""",1
1,"""00066be8e20318869c38c66be466631a_420""",11.176292,0.279881,-11.625697,-44.0,0.0,false,0.0,false,"""drive""",false,true,4.970268,-0.007936,0.005028,10.350489,-0.032374,-0.020701,15.770054,0.084073,0.008645,21.132415,0.391343,0.036335,26.316489,0.843124,0.065,31.383814,1.42507,0.073083,"""00066be8e20318869c38c66be466631a""",1
2,"""00066be8e20318869c38c66be466631a_520""",10.472548,0.231099,-2.985105,-132.0,0.0,false,0.18,true,"""drive""",false,false,4.815701,-0.000813,0.017577,10.153522,-0.0278,0.026165,15.446539,-0.155987,0.040397,20.61816,-0.356932,0.058765,25.677387,-0.576985,0.102859,30.460033,-0.841894,0.152889,"""00066be8e20318869c38c66be466631a""",1
3,"""000fb056f97572d384bae4f5fc1e0f28_120""",6.055565,-0.117775,7.632668,173.0,0.0,false,0.0,false,"""drive""",false,false,2.812608,0.033731,0.0059,5.975378,0.137848,0.01621,9.186793,0.322997,0.031626,12.37311,0.603145,0.031858,15.703514,0.960717,0.043479,19.311182,1.374655,0.058754,"""000fb056f97572d384bae4f5fc1e0f28""",3
4,"""000fb056f97572d384bae4f5fc1e0f28_20""",3.316744,1.276733,-31.725477,-114.0,0.0,false,0.255,true,"""drive""",false,false,1.55186,-0.041849,-0.008847,3.675162,-0.125189,-0.013725,6.113567,-0.239161,-0.012887,8.770783,-0.381813,-0.003898,11.619313,-0.554488,0.011393,14.657048,-0.7788,0.044243,"""000fb056f97572d384bae4f5fc1e0f28""",3


### データ拡張


In [4]:
def get_train_transform():
    return A.ReplayCompose(
        [
            # 1. 基本的なサイズ調整（必須）
            A.Resize(cfg.cnn.size, cfg.cnn.size),
            # 2. 軽量な明るさ調整（高速で効果が高い）
            A.ColorJitter(
                brightness=0.2,
                contrast=0.2,
                saturation=0.1,
                hue=0.0,  # 色相の変更は省略
                p=0.7,
            ),
            # # 3. 軽量なブレ表現（1つだけ選択）
            # A.OneOf(
            #     [
            #         A.MotionBlur(blur_limit=3),
            #         A.GaussianBlur(blur_limit=3),
            #     ],
            #     p=0.3,
            # ),
            # # 4. 軽量なノイズ（シンプルな設定）
            # A.GaussNoise(
            #     var_limit=(10, 30),
            #     mean=0,
            #     per_channel=False,  # チャンネル共通で処理
            #     p=0.3,
            # ),
            # 5. 正規化（必須）
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]
    )


def get_valid_transform():
    return A.ReplayCompose(
        [
            A.Resize(cfg.cnn.size, cfg.cnn.size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]
    )


### データセット

In [5]:
class CustomDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, is_train=True):
        self.df = df
        self.img_dir = Path(img_dir)
        self.is_train = is_train

        # デフォルトの変換処理
        if transform is None:
            self.transform = A.ReplayCompose(
                [
                    A.Resize(cfg.cnn.size, cfg.cnn.size),
                    A.Normalize(
                        mean=[0.485, 0.456, 0.406],  # 通常のImageNet平均値
                        std=[0.229, 0.224, 0.225],  # 通常のImageNet標準偏差
                    ),
                    ToTensorV2(),
                ]
            )
        else:
            self.transform = transform

        self.target_cols = cfg.target_cols

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df[idx]
        img_folder = self.img_dir / row["ID"].item()

        # 3枚の画像を読み込み
        img_names = ["image_t-1.0.png", "image_t-0.5.png", "image_t.png"]
        imgs = []

        for img_name in img_names:
            img_path = img_folder / img_name
            img = cv2.imread(str(img_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgs.append(img)

        if self.transform:
            replay = None
            transformed_imgs = []

            for img in imgs:
                if replay is None:
                    # 最初の画像に対して変換を実行し、replayを保存
                    transformed = self.transform(image=img)
                    replay = transformed["replay"]
                else:
                    # 2枚目以降は保存したreplayを使用
                    transformed = A.ReplayCompose.replay(replay, image=img)
                transformed_imgs.append(transformed["image"])

        # チャネル方向に結合 (C*3, H, W)
        img_tensor = torch.cat(transformed_imgs, dim=0)

        # ターゲットの準備
        if self.is_train:
            target = torch.tensor(row[self.target_cols].to_numpy(), dtype=torch.float32).squeeze(0)
            return img_tensor, target
        else:
            return img_tensor


### モデル

In [6]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_trainable=True):
        super().__init__()
        if p_trainable:
            self.p = nn.Parameter(torch.ones(1) * p)
        else:
            self.p = p
        self.eps = eps

    def forward(self, x):
        x = x.clamp(min=self.eps).pow(self.p)
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.pow(1.0 / self.p)
        return x


In [7]:
class CustomModel(nn.Module):
    def __init__(self, cfg, pretrained=False, target_size=None, model_name=None):
        super().__init__()

        # エンコーダーとプーリング層
        self.encoder = timm.create_model(
            cfg.model_name, pretrained=cfg.pretrained, num_classes=0, in_chans=cfg.in_chans
        )
        self.n_features = self.encoder.num_features  # 例：1280 (EfficientNet-B0の場合)
        # self.pool = GeM(p=3, eps=1e-6, p_trainable=True)

        # Transformerレイヤー
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.n_features,  # 特徴量の次元数
            nhead=8,  # アテンションヘッド数
            dim_feedforward=self.n_features * 4,
            dropout=0.1,
            batch_first=True,
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=1)

        # 位置エンコーディング: (1, 3, n_features)
        # 3は時系列の長さ（3つの時点）
        self.pos_embedding = nn.Parameter(torch.randn(1, 3, self.n_features))

        self.target_size = cfg.target_size if target_size is None else target_size
        self.fc = nn.Sequential(
            nn.Linear(self.n_features * 3, self.n_features * 2),
            nn.LayerNorm(self.n_features * 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(self.n_features * 2, self.target_size),
        )

    def forward(self, x):
        # 入力 x: (batch_size, 9, H, W)
        # 9 = 3チャネル × 3時点
        batch_size = x.size(0)

        # 3フレームに分割
        # 各frame: (batch_size, 3, H, W)
        frames = x.chunk(3, dim=1)

        # 各フレームから特徴抽出
        features = []
        for frame in frames:
            # encoder出力: (batch_size, n_features, h, w)
            feat = self.encoder(frame)
            # pool後: (batch_size, n_features)
            # feat = self.pool(feat).reshape(batch_size, -1)
            features.append(feat)

        # 時系列データとしてスタック
        # sequence: (batch_size, 3, n_features)
        sequence = torch.stack(features, dim=1)

        # 位置エンコーディングを追加
        # sequence: (batch_size, 3, n_features)
        sequence = sequence + self.pos_embedding

        # Transformer処理
        # sequence: (batch_size, 3, n_features)
        sequence = self.transformer(sequence)

        # 全結合層
        # reshape: (batch_size, 3 * n_features)
        # 最終出力: (batch_size, target_size)
        out = self.fc(sequence.reshape(batch_size, -1))

        return out


### 学習

In [8]:
DEBUG = False

if DEBUG:
    cfg.cnn.num_epochs = 1
    train = train.head(100)


In [9]:
# 実験結果格納用のディレクトリを作成
japan_tz = pytz.timezone("Asia/Tokyo")
cfg.run_time = datetime.now(japan_tz).strftime("%Y%m%d_%H%M%S")
create_dir(cfg.data.results_path)

# CV用の配列を初期化
oof_predictions = np.zeros((len(train), len(cfg.target_cols)))
models = {}

for fold, (train_idx, valid_idx) in enumerate(gkf.split(train, groups=train["scene"])):
    print(f"Fold {fold + 1}")

    # データセットの作成
    train_dataset = CustomDataset(train[train_idx], cfg.data.img_root, transform=get_train_transform())
    valid_dataset = CustomDataset(train[valid_idx], cfg.data.img_root, transform=get_valid_transform())

    train_loader = DataLoader(train_dataset, batch_size=cfg.cnn.batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=cfg.cnn.batch_size, shuffle=False)

    # モデル、損失関数、オプティマイザーの初期化
    model = CustomModel(cfg.cnn).to(device)
    criterion = nn.HuberLoss()
    # criterion = nn.L1Loss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.cnn.lr)
    total_steps = len(train_loader) * cfg.cnn.num_epochs
    scheduler = get_cosine_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=total_steps * 0.1,
        num_training_steps=total_steps,
    )

    best_loss = float("inf")

    # 学習ループ
    for epoch in range(cfg.cnn.num_epochs):
        model.train()
        for images, targets in tqdm(train_loader):
            images = images.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            scheduler.step()

        # 検証
        model.eval()
        valid_losses = []
        with torch.no_grad():
            for images, targets in valid_loader:
                images = images.to(device)
                targets = targets.to(device)
                outputs = model(images)
                loss = criterion(outputs, targets)
                valid_losses.append(loss.item())

        valid_loss = np.mean(valid_losses)
        print(f"Epoch {epoch + 1}, Valid Loss: {valid_loss:.4f}")

        # ベストモデルの保存
        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save(model.state_dict(), f"{cfg.data.results_path}/model_fold{fold}.pth")

    # ベストモデルでOOF予測
    model.load_state_dict(torch.load(f"{cfg.data.results_path}/model_fold{fold}.pth"))
    model.eval()

    valid_dataset = CustomDataset(train[valid_idx], cfg.data.img_root, transform=get_valid_transform())
    valid_loader = DataLoader(valid_dataset, batch_size=cfg.cnn.batch_size, shuffle=False)

    with torch.no_grad():
        for i, (images, _) in enumerate(valid_loader):
            images = images.to(device)
            outputs = model(images)
            start_idx = i * cfg.cnn.batch_size
            end_idx = start_idx + outputs.shape[0]
            oof_predictions[valid_idx[start_idx:end_idx]] = outputs.cpu().numpy()

# CVスコアの計算（MAEの平均）
mae_scores = []
for i in range(len(cfg.target_cols)):
    mae = np.mean(np.abs(oof_predictions[:, i] - train[cfg.target_cols[i]].to_numpy()))
    mae_scores.append(mae)

cv_score = np.mean(mae_scores)
print(f"CV Score: {cv_score:.4f}")

# oofを保存
np.save(f"{cfg.data.results_path}/oof_predictions.npy", oof_predictions)


Directory created: ../../results/005/20241121_232428
Fold 1


/home/marumarukun/pj/compe/atma_18/.venv/lib/python3.12/site-packages/pydantic/main.py:212: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/home/marumarukun/pj/compe/atma_18/.venv/lib/python3.12/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(
 15%|█▌        | 168/1085 [01:19<06:59,  2.19it/s]

In [ ]:
# testの推論
test_dataset = CustomDataset(test, cfg.data.img_root, transform=get_valid_transform(), is_train=False)
test_loader = DataLoader(test_dataset, batch_size=cfg.cnn.batch_size, shuffle=False)

# 5fold分の予測値を格納する配列
test_predictions = np.zeros((len(test), len(cfg.target_cols), cfg.n_splits))

# 各foldのモデルで予測
for fold in range(cfg.n_splits):
    print(f"Predicting using fold {fold + 1} model")

    # モデルの読み込み
    model = CustomModel(cfg.cnn).to(device)
    model.load_state_dict(torch.load(f"{cfg.data.results_path}/model_fold{fold}.pth"))
    model.eval()

    fold_predictions = []
    with torch.no_grad():
        for images in tqdm(test_loader):
            images = images.to(device)
            outputs = model(images)
            fold_predictions.append(outputs.cpu().numpy())

    # バッチごとの予測を結合
    fold_predictions = np.concatenate(fold_predictions, axis=0)
    test_predictions[:, :, fold] = fold_predictions

# 5fold分の予測値の平均を計算
final_predictions = test_predictions.mean(axis=2)

# submissionファイルの作成
exprs = [pl.Series(final_predictions[:, i]).alias(cfg.target_cols[i]) for i in range(len(cfg.target_cols))]
submission = sample_submission.with_columns(exprs)
submission.write_csv(f"{cfg.data.results_path}/submission.csv")
print("Submission file created!")

# 確認
submission.head()


In [13]:
# final_predictionsを保存
np.save(f"{cfg.data.results_path}/final_predictions.npy", final_predictions)


In [ ]:
final_predictions.shape
